In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema.runnable import RunnablePassthrough


llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.8,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=10,
)

examples = [
    {
        "question": "I want a summary of the movie 'Top Gun' only written by emojis.",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "question": "I want a summary of the movie 'The Godfather' only written by emojis.",
        "answer": "👨‍👨‍👦🔫🍝",
    },
    {
        "question": "I want a summary of the movie 'Interstellar' only written by emojis.",
        "answer": "🧑‍🚀🚀⭐",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "{answer}"),
    ]
)

chat_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a movie lover. Basically, when given question about a movie, you reply with exactly 3 emoji that represents the movie well. But when asked the movie that you answered before, you must reply with the same 3 emoji that you answered before. Also, when human asks you about movies that you remember, you reply every movie title that you remember.",
        ),
        # example을 MessagesPlaceholder보다 먼저 주면 AI가 history보다 example을 더 중요시에서 example을 토대로 답할 수도 있다
        MessagesPlaceholder(variable_name="history"),
        chat_prompt,
        (
            "human",
            "{question}",
        ),
    ]
)


def save(input, output):
    memory.save_context(
        {"input": input},
        {"output": output},
    )


save("I want a summary of the movie 'Top Gun' only written by emojis.", "🛩️👨‍✈️🔥")
save(
    "I want a summary of the movie 'The Godfather' only written by emojis.",
    "👨‍👨‍👦🔫🍝",
)
save("I want a summary of the movie 'Interstellar' only written by emojis.", "🧑‍🚀🚀⭐")


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm


def invoke_by_movie(movie):
    question = f"I want a summary of the movie '{movie}' only written by emojis."
    result = chain.invoke({"question": question})
    save(question, result.content)


def invoke_by_question(question):
    result = chain.invoke({"question": question})
    save(question, result.content)

In [4]:
invoke_by_movie("Avengers: Endgame")

🦸‍♂️🦸‍♀️⚔️

In [5]:
invoke_by_movie("Dune")

🌌🏜️🪐

In [6]:
invoke_by_question("Do you remember the movies I asked before?")

Yes! Here are the movies you asked about:

1. Top Gun
2. The Godfather
3. Interstellar
4. Avengers: Endgame
5. Dune

In [7]:
invoke_by_movie("Interstellar")

🧑‍🚀🚀⭐

In [8]:
invoke_by_movie("Top Gun")

🛩️👨‍✈️🔥

In [9]:
invoke_by_movie("The Godfather")

👨‍👨‍👦🔫🍝

In [10]:
invoke_by_movie("The Amazing Spider-man")

🕷️🕸️🕺

In [12]:
invoke_by_question("Do you remember the movies I asked before?")

Yes! Here are the movies you asked about:

1. Top Gun
2. The Godfather
3. Interstellar
4. Avengers: Endgame
5. Dune
6. The Amazing Spider-man